# mapping NJT bus routes using Clever Devices API route XML

# global settings

In [4]:
# gmaps tutorial https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html

In [5]:
source = 'nj'
rt = 119
direction = 0 # inbound

In [6]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)



In [7]:
import sys
import src.lib.BusAPI as Buses
import gmaps
import config

gmaps.configure(api_key=config.free_maps_api_key)

In [23]:
# return a list of routes, by direction (doubled)

routedata=Buses.parse_xml_getRoutePoints(Buses.get_xml_data(source, 'routes', route=rt))


In [15]:
# new way
route_list = []
for i in routedata:
    stop_list = []
    point_list = []
    for path in i.paths:
        stops_points = []
        route_points = []
        for point in path.points:
            if isinstance(point, Buses.Route.Stop):
                stops_points.append(point)
            else:
                route_points.append(point)

        stop_list.append(stops_points)
        point_list.append(route_points)

    route_list.append(stop_list)
    route_list.append(point_list)

# route_stop_list = route_list[:2] # chop off the duplicate half


In [16]:
route_stop_list

[[[<src.lib.BusAPI.Stop instance at 0x1074d6b00>,
   <src.lib.BusAPI.Stop instance at 0x1074e2830>,
   <src.lib.BusAPI.Stop instance at 0x1074e2b00>,
   <src.lib.BusAPI.Stop instance at 0x1074e2b90>,
   <src.lib.BusAPI.Stop instance at 0x1074e2c20>,
   <src.lib.BusAPI.Stop instance at 0x1074e2cb0>,
   <src.lib.BusAPI.Stop instance at 0x1074e2d40>,
   <src.lib.BusAPI.Stop instance at 0x1074e2ef0>,
   <src.lib.BusAPI.Stop instance at 0x1074e2f80>,
   <src.lib.BusAPI.Stop instance at 0x1074e5050>,
   <src.lib.BusAPI.Stop instance at 0x1074e50e0>,
   <src.lib.BusAPI.Stop instance at 0x1074e5170>,
   <src.lib.BusAPI.Stop instance at 0x1074e5200>,
   <src.lib.BusAPI.Stop instance at 0x1074e5290>,
   <src.lib.BusAPI.Stop instance at 0x1074e5320>,
   <src.lib.BusAPI.Stop instance at 0x1074e53b0>,
   <src.lib.BusAPI.Stop instance at 0x1074e5440>,
   <src.lib.BusAPI.Stop instance at 0x1074e54d0>,
   <src.lib.BusAPI.Stop instance at 0x1074e5560>,
   <src.lib.BusAPI.Stop instance at 0x1074e55f0>,


In [17]:
m = gmaps.Map()

m.add_layer(gmaps.symbol_layer([(float(p.lat), float(p.lon)) for p in route_points], fill_color='green', stroke_color='green', scale=2))
m.add_layer(gmaps.symbol_layer([(float(p.lat), float(p.lon)) for p in stops_points], fill_color='red', stroke_color='red', scale=5))
m

TWFwKGNvbmZpZ3VyYXRpb249eydhcGlfa2V5JzogdSdBSXphU3lCOFNIT0VhaW9qTlZ4QWRnMjQ2NlktdWpua2UwWWZlUkUnfSwgZGF0YV9ib3VuZHM9Wyg0MC42NDU4ODc1NzI3NTcyNCwgLTfigKY=


In [19]:
# 2.stops only 

# separate figures for route points and stop points (Alex original version) - direction inbound
route_points = []
stops_points = []

for i in routedata[0][direction].points:
    if isinstance(i, Buses.Route.Stop):
        stops_points.append(i)
    else:
        route_points.append(i)
            
m = gmaps.Map()

#m.add_layer(gmaps.symbol_layer([(float(p.lat), float(p.lon)) for p in route_points], fill_color='green', stroke_color='green', scale=2))
m.add_layer(gmaps.symbol_layer([(float(p.lat), float(p.lon)) for p in stops_points], fill_color='red', stroke_color='red', scale=5))
m

AttributeError: Route instance has no attribute '__getitem__'

In [20]:
# 3. map - simple start-end route, no waypoints

# set up the directions-eligible API key
import config
gmaps.configure(api_key=config.paid_directions_api_key)


# separate figures for route points and stop points (Alex original version)
route_points = []
stops_points = []

for i in routes[0][direction].points:
    if isinstance(i, Buses.Route.Stop):
        stops_points.append(i)
    else:
        route_points.append(i)

end_num = (len(stops_points) - 1)
start = (float(stops_points[0].lat), float(stops_points[0].lon))
end = (float(stops_points[end_num].lat), float(stops_points[end_num].lon))

fig = gmaps.figure()
start2end = gmaps.directions_layer(start, end)
fig.add_layer(start2end)
fig



NameError: name 'routes' is not defined

In [22]:
# 4. map - waypoints map 

# set up the directions-eligible API key
import config
gmaps.configure(api_key=config.paid_directions_api_key)

# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

# separate figures for route points and stop points (modified from Alex original version)
way_points_full = []

# no longer distinguish between path stops and points from the XML
for i in routedata[0][direction].points:
    way_points_full.append(i)

# slice out the waypoints for mapping
number_of_way_points=10 # user configurable up to 24 (maybe limit to 23 since we prepend the start point below)
                        # (per https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#directions-layer)



way_points_bins = list(chunks(way_points_full,(len(way_points_full)/number_of_way_points)))

# way_points_bins

AttributeError: Route instance has no attribute '__getitem__'

In [ ]:
way_points = []
way_points.append(way_points_full[0])  # add the starting point
for i in way_points_bins:
    way_points.append(i[-1])
    
way_points_tuples=[]
for wp in way_points:
    way_points_tuples.append((float(wp.lat),float(wp.lon)))

# way_points_tuples

In [ ]:
# make the map 

import config
gmaps.configure(api_key=config.paid_directions_api_key)

end_num = len(way_points)-1
start = (float(way_points[0].lat), float(way_points[0].lon))
end = (float(way_points[end_num].lat), float(way_points[end_num].lon))

waypoint_map = gmaps.figure()
points_layer = gmaps.directions_layer(
        start, end, waypoints=[x for x in way_points_tuples])
waypoint_map.add_layer(points_layer)
waypoint_map

In [ ]:
# OLD METHOD BROKEN

# separate figures for route points and stop points (Alex original version) - direction inbound
route_points = []
stops_points = []

for i in routes[0][direction].points:
    if isinstance(i, Buses.Route.Stop):
        stops_points.append(i)
    else:
        route_points.append(i)
       